# reference data fips

This is a text cell. Start editing!

In [1]:
import org.jsoup._
import scala.collection.JavaConverters

val url = "https://www.nrcs.usda.gov/wps/portal/nrcs/detail/?cid=nrcs143_013696"

val doc = Jsoup.connect(url).get

val statesTbl = doc.select("#detail > table").select("td")

statesTbl

<td> Alabama</td>
<td> AL</td>
<td> 01</td>
<td> Alaska</td>
<td> AK</td>
<td> 02</td>
<td> Arizona</td>
<td> AZ</td>
<td> 04</td>
<td> Arkansas</td>
<td> AR</td>
<td> 05</td>
<td> California</td>
<td> CA</td>
<td> 06</td>
<td> Colorado</td>
<td> CO</td>…

In [2]:
val stateRef =statesTbl.toArray.map(_.asInstanceOf[org.jsoup.nodes.Element])
    .map(_.text).grouped(3).map(row => (row(0), row(1), row(2).toInt).productIterator.toSeq).toSeq

stateRef    

Stream(Stream(Alabama, ?), ?)

In [3]:
{"type":"table","value":"Out","rowRange":[0,20]}

In [4]:
import java.nio.file.{Files, Paths}
Files.writeString(Paths.get("states.csv"), stateRef.mkString("\n") )





states.csv

In [5]:
import java.util.Properties
import org.apache.kafka.clients.producer._

def writeToKafka(topic: String, key: String, value: String): Unit = {
    val props = new Properties()
    props.put("bootstrap.servers", "localhost:9092")
    props.put("key.serializer", "org.apache.kafka.common.serialization.StringSerializer")
    props.put("value.serializer", "org.apache.kafka.common.serialization.StringSerializer")
    //props.put("compression.type", "snappy")

    val producer = new KafkaProducer[String, String](props)
    val record = new ProducerRecord[String, String](topic, key, value)

    stateRef.foreach(record => producer.send(new ProducerRecord[String, String]("test.states", "a", record.mkString(","))))

    producer.close()
}

writeToKafka("test.lars", "123", "Something really important")